In [2]:
import pandas as pd

wine = pd.read_csv("https://bit.ly/wine_csv_data")

input_data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']


##### Validation Set
- 모델 학습 이후의 성능 평가는 검증셋을 통해 한다.

In [5]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    input_data, target, test_size=0.2, random_state=42
)

print(train_input.shape)
print(test_input.shape)

# 훈련셋에서 검증셋을 다시 분리
train_input, val_input, train_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)
print('--Make Validation Set--')
print(train_input.shape)
print(val_input.shape)

(5197, 3)
(1300, 3)
--Make Validation Set--
(4157, 3)
(1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(train_input, train_target)
print(dt.score(train_input, train_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.8615384615384616


👉 훈련셋에 오버피팅되어있을 확률이 높음

- 매개변수를 바꿔서 좀 더 좋은 모델을 찾아야 한다.

---
# 교차 검증(Cross Validation)
- 뭐가 됐든 일단 훈련 데이터를 많이 사용해야 좋은 모델이 만들어짐.
    - 검증셋을 만드느라 훈련 데이터가 줄어들었음
    - 그렇다고 검증셋의 양을 줄이면 측정하는 모델의 성능이 들쑥날쑥해질것임
    - 이를 해결하기 위한 방법이 검증셋의 양은 그대로 두고, 서로 다른 여러 검증셋을 통해 모델 성능을 측정하는 방식(Cross Validation)
        - K-Fold Cross Validation: 훈련 셋을 K개로 쪼개어, K개의 검증셋을 순차적으로 각각의 검증셋으로 활용하는 방식

In [7]:
from sklearn.model_selection import cross_validate

train_input, test_input, train_target, test_target = train_test_split(
    input_data, target, test_size=0.2, random_state=42
)

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.0049839 , 0.00494504, 0.00574613, 0.00542593, 0.00481677]), 'score_time': array([0.00095701, 0.00076795, 0.00081897, 0.0009253 , 0.00096798]), 'test_score': array([0.86826923, 0.85288462, 0.87006737, 0.85466795, 0.84119346])}


- 각 value값이 5개씩 있는것으로 볼 때, 기본 폴드값은 5라는것을 알 수 있음
- `fit_time`은 훈련시간, `score_time`은 검증하는 시간을 의미함
- `test_score`는 각각의 테스트셋에 대한 점수로, 이 값의 평균값이 모델의 최종 성능임

In [8]:
import numpy as np

print(np.mean(scores['test_score']))

0.8574165247649367


- 주의해야하는 점은, `cross_validate()`는 훈련셋을 섞어서 폴드를 나누는게 아니라, 그냥 있는대로 나눈다.
- 앞선 `train_test_split()`은 처음에 데이터를 섞고 나서 데이터를 분리했기 때문에 상관 없는데, 교차 검증에서 데이터를 섞기 위해서는 분할기(splitter)를 지정해야한다.
    - `cross_validate()`함수는 일반적으로 회귀 모델의 경우 `KFold`분할기, 분류 모델의 경우 타겟 클래스를 골고루 나누기 위해 `StratifiedKFold`를 사용한다. 사용은 아래와 같다.

In [9]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8587641593247947


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.854726915666222


---
# 하이퍼 파라미터 튜닝
- 모델마다 여러 하이퍼 파라미터값이 존재하는데, 각 하이퍼 파라미터값에 대한 모델의 성능은 하이퍼 파라미터들의 복합적인 관계를 통해 좌우되기때문에 복잡함
- 사이킷런에서는 최적의 하이퍼 파라미터 조합을 찾기 위한 클래스로 그리드서치(`GridSearch`)를 제공